In [6]:
!pip install feedparser


In [7]:
from google.oauth2 import service_account

auth.authenticate_user()
credentials = service_account.Credentials.from_service_account_info(
    'here you have declara you account credentials',
    scopes=['https://www.googleapis.com/auth/spreadsheets']
)
service = build('sheets', 'v4', credentials=credentials)


In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from datetime import datetime
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from google.colab import auth
from googleapiclient.discovery import build
from google.oauth2 import service_account

#web scraping CNBC data
def scrape_cnbc(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    headlines = [headline.text.strip() for headline in soup.find_all('h3', class_='SearchResultCard-headline')]
    return headlines

#google rss feed news
def get_google_news_headlines(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    headlines = [item.text for item in soup.find_all('title')]
    return headlines[2:]

#sentiment analysis
def analyze_sentiment(text):
    classifier = pipeline("sentiment-analysis")
    result = classifier(text)
    return result[0]['label'], result[0]['score']

#extracting the organizations names
def extract_organizations(text):
    ner_classifier = pipeline("ner")
    entities = ner_classifier(text)
    organizations = [entity['entity'] for entity in entities if entity['entity'] != 'O' and entity['entity'] != 'PER']
    return organizations


#Headline and sentiment analysis
def fetch_headlines_and_sentiments():
    cnbc_headlines = scrape_cnbc("https://www.cnbc.com/search/?query=green%20hydrogen&qsearchterm=green%20hydrogen")
    google_news_headlines = get_google_news_headlines("https://news.google.com/rss/search?q=green%20hydrogen&hl=en-IN&gl=IN&ceid=IN:en")
    all_headlines = cnbc_headlines + google_news_headlines
    headlines_data = []

    for headline in all_headlines:
        sentiment_label, sentiment_score = analyze_sentiment(headline)
        organizations = extract_organizations(headline)
        headlines_data.append({'Headline': headline, 'Sentiment': sentiment_label, 'Sentiment Score': sentiment_score, 'Organizations': organizations})

    df = pd.DataFrame(headlines_data)
    df['News Date'] = datetime.now().strftime("%Y-%m-%d")
    return df

def write_to_google_sheets(df, spreadsheet_id):
    auth.authenticate_user()
    credentials = service_account.Credentials.from_service_account_info(
        'account info needed to be declared',
        scopes=['https://www.googleapis.com/auth/spreadsheets']
    )
    service = build('sheets', 'v4', credentials=credentials)
    range_name = 'Sheet1!A1'
    data = df.to_csv(index=False).split('\n')

    body = {'values': [data]}
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=range_name,
        valueInputOption='RAW',
        body=body
    ).execute()

def plot_sentiment_trend(data):
    data['News Date'] = pd.to_datetime(data['News Date'])
    data['Week'] = data['News Date'].dt.strftime('%U-%Y')
    weekly_sentiment = data.groupby('Week')['Sentiment Score'].mean()

    plt.figure(figsize=(10, 6))
    weekly_sentiment.plot(kind='line', marker='o', color='b')
    plt.title('Week Wise Trend of Average Sentiment Score')
    plt.xlabel('Week')
    plt.ylabel('Average Sentiment Score')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def generate_wordcloud(data):
    organizations = ' '.join(data['Organizations'].explode().dropna())
    wordcloud = WordCloud(width=800, height=400, background_color='white', min_font_size=10).generate(organizations)

    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Organization Names in News Headlines')
    plt.show()

# fetch data
data = fetch_headlines_and_sentiments()

data.to_csv('green_hydrogen_news.csv', index=False)

# data to Google Sheets
write_to_google_sheets(data, 'https://docs.google.com/spreadsheets/d/1uSfU_O-COBiieyVuMoqHYi0Pso_QbXbWygI495Bkfg8/edit#gid=0')

# Plot
plot_sentiment_trend(data)

# word cloud
generate_wordcloud(data)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).

AttributeError: module 'google.auth.credentials' has no attribute 'with_scopes'